## Previsão de vendas: Use dados históricos de vendas de uma empresa para criar um modelo de regressão capaz de prever as vendas futuras. Isso ajudará a empresa a planejar sua produção, estoque e estratégias de marketing com base nas previsões geradas pelo modelo. ##

In [1]:
#Bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#
# É possível utilizar a biblioteca "os" para salvar o caminho da pasta (com isto, é possível fazer uma verificação inicial)
# --------------------------------------------------------------------------------------------------------------------------------------------
# import os

# caminho_pasta = '/caminho/para/a/pasta'  # Substitua pelo endereço da pasta desejada

# # Verifique se o caminho é válido
# if os.path.exists(caminho_pasta):
#     # Transforme o endereço da pasta em uma variável
#     variavel_pasta = os.path.abspath(caminho_pasta)
#     print("Variável da pasta:", variavel_pasta)
# else:
#     print("O caminho da pasta não é válido.")
# --------------------------------------------------------------------------------------------------------------------------------------------


In [2]:
# leitura do dataset, disponível em: https://www.kaggle.com/datasets/kreeshrajani/used-car-price-dataset?resource=download
data_raw=pd.read_csv('used_car_dataset.csv')  
data_raw  

,car_name,car_price_in_rupees,kms_driven,fuel_type,city,year_of_manufacture
0,Hyundai Grand i10 Magna 1.2 Kappa VTVT [2017-2...,₹ 4.45 Lakh,"22,402 km",Petrol,Mumbai,2016
1,Maruti Suzuki Alto 800 Lxi,₹ 2.93 Lakh,"10,344 km",Petrol,Kolkata,2019
2,Tata Safari XZ Plus New,₹ 22.49 Lakh,"12,999 km",Diesel,Bangalore,2021
3,Maruti Suzuki Ciaz ZXI+,₹ 6.95 Lakh,"45,000 km",Petrol,Thane,2016
4,Jeep Compass Sport Plus 1.4 Petrol [2019-2020],₹ 12 Lakh,"11,193 km",Petrol,Kolkata,2019
...,...,...,...,...,...,...
2100,Ford Figo Titanium1.5 TDCi,₹ 3.6 Lakh,"42,158 km",Diesel,Kolkata,2015
2101,MINI Cooper Countryman Cooper D,₹ 22 Lakh,"68,862 km",Diesel,Hyderabad,2013
2102,Hyundai Verna 1.6 VTVT SX,₹ 8.38 Lakh,"37,622 km",Petrol,Chennai,2018
2103,Maruti Suzuki Ciaz VXi+ AT,₹ 6.75 Lakh,"64,726 km",Petrol,Mumbai,2017


In [3]:
# EDA para encontrar irregularidades nos dados
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105 entries, 0 to 2104
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   car_name             2105 non-null   object
 1   car_price_in_rupees  2105 non-null   object
 2   kms_driven           2105 non-null   object
 3   fuel_type            2105 non-null   object
 4   city                 2105 non-null   object
 5   year_of_manufacture  2105 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 98.8+ KB


In [4]:
data_raw.isna().sum()
# no dataset, podemos verificar que não possuem dados nulos, porém dados que eram para ser numéricos, são do tipo object.
# então, devemos transformar as colunas em numéricas, para realizar a modelagem a seguir.

car_name               0
car_price_in_rupees    0
kms_driven             0
fuel_type              0
city                   0
year_of_manufacture    0
dtype: int64

In [5]:
data_raw['car_price_in_rupees'].sort_values(ascending=True)
# no dataset, existem 2 tipos de unidades, o Lakh, que vale 100.000, e o Crore, que vale 10 milhões.

376       ₹ 1.1 Lakh
21      ₹ 1.15 Crore
1340    ₹ 1.19 Crore
313     ₹ 1.23 Crore
1850    ₹ 1.36 Crore
            ...     
1451     ₹ 9.99 Lakh
804      ₹ 9.99 Lakh
566        ₹ 93 Lakh
17         ₹ 95 Lakh
1635        ₹ 99,991
Name: car_price_in_rupees, Length: 2105, dtype: object

In [6]:
# Realizando o tratamento necessario para separar colunas numericas e categoricas

# Transformando a vírgula do menor valor da coluna de preço do carro.
data_raw['car_price_in_rupees'] = data_raw['car_price_in_rupees'].str.replace(',','')

#Função que transforma os textos (Lakh e Crore) nas unidades e então multiplica pelo valor numérico.
def converter_valor_monetario(valor):

    '''função que transforma as unidades "Crore"= 10 milhões e "Lakh"= 100 mil na moeda da India rúpia'''

    if 'Crore' in valor:
        valor_numerico = float(valor.split()[1]) * 10000000
    elif 'Lakh' in valor:
        valor_numerico = float(valor.split()[1]) * 100000
    else:
        valor_numerico = float(valor.split()[1])
    return valor_numerico

data_raw['car_price_in_rupees']= [converter_valor_monetario(valor) for valor in data_raw['car_price_in_rupees'].values ]

In [7]:
# Dados após as transformações

data_raw['car_price_in_rupees'].sort_values(ascending=True)

499        60000.0
1635       99991.0
376       110000.0
1656      142000.0
788       150000.0
           ...    
21      11500000.0
1340    11900000.0
313     12300000.0
1850    13600000.0
1392    19500000.0
Name: car_price_in_rupees, Length: 2105, dtype: float64

In [8]:
# Removendo as vírgulas, e a unidade de medida, e transformando a coluna de Kms rodados em float.

data_raw['kms_driven'] = data_raw['kms_driven'].str.replace(',','')
data_raw['kms_driven'] = data_raw['kms_driven'].str.replace(' km','')
data_raw['kms_driven'] = data_raw['kms_driven'].astype(float)
data_t1=data_raw.copy()

In [9]:
# separando dados categóricos e numéricos
data_cat = data_t1.select_dtypes(exclude='number')
data_num = data_t1.select_dtypes(exclude='object')
data_cat

,car_name,fuel_type,city
0,Hyundai Grand i10 Magna 1.2 Kappa VTVT [2017-2...,Petrol,Mumbai
1,Maruti Suzuki Alto 800 Lxi,Petrol,Kolkata
2,Tata Safari XZ Plus New,Diesel,Bangalore
3,Maruti Suzuki Ciaz ZXI+,Petrol,Thane
4,Jeep Compass Sport Plus 1.4 Petrol [2019-2020],Petrol,Kolkata
...,...,...,...
2100,Ford Figo Titanium1.5 TDCi,Diesel,Kolkata
2101,MINI Cooper Countryman Cooper D,Diesel,Hyderabad
2102,Hyundai Verna 1.6 VTVT SX,Petrol,Chennai
2103,Maruti Suzuki Ciaz VXi+ AT,Petrol,Mumbai


In [10]:
data_num

,car_price_in_rupees,kms_driven,year_of_manufacture
0,445000.0,22402.0,2016
1,293000.0,10344.0,2019
2,2249000.0,12999.0,2021
3,695000.0,45000.0,2016
4,1200000.0,11193.0,2019
...,...,...,...
2100,360000.0,42158.0,2015
2101,2200000.0,68862.0,2013
2102,838000.0,37622.0,2018
2103,675000.0,64726.0,2017


In [11]:
# Criando os pipelines

pipe_reg = Pipeline([('scaler', StandardScaler())
                     ('model', [LinearRegression(), LogisticRegression()])
                    ])

<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
C:\Users\vitor\AppData\Local\Temp/ipykernel_21120/1979628042.py:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  pipe_reg = Pipeline([('scaler', StandardScaler())


TypeError: 'tuple' object is not callable